# Notebook to extract statistics from DataHub

In this notebook, we will explore how to get stats from DataHub, specifically for DataHub views.

For this example we will be using vDataFeed_UCube_AssetsCompany, ingested locally. To use this for on-premise entities change the config file accordingly.

In [ ]:
# Set the urn of the entity here
urn = 'urn:li:dataset:(urn:li:dataPlatform:mssql,server.db.schema.table,DEV)'


In [ ]:
import logging
import configparser

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

config = configparser.ConfigParser()
config.read('src/config.conf')

# Set the environment variables we need

gms_server = config['DATAHUB']['gms_server']
base_url = config['DATAHUB']['base_url']
token = config['DATAHUB']['token']

In [ ]:
# GET Table DatasetFieldProfile using GraphiQL

import requests

endpoint = f'{gms_server}/api/graphql'

query = f"""
query {{
  dataset(urn:"{urn}")
  {{
    datasetProfiles{{
      fieldProfiles{{
        fieldPath
        uniqueCount
        uniqueProportion
        nullCount
        min
        max
        mean
        median
        stdev
        sampleValues
      }}
    }}
  }}
}}"""

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

response = requests.post(endpoint, 
                         headers=headers,
                         json={'query': query})


In the response (as you can see below), there is the information for each column (fieldPath), and its related stats values

In [ ]:
response.json()

Below we extract this information as a pandas df.

First, let's to it for min and max.

In [ ]:
import pandas as pd

data = response.json()

# Extract the data
field_profiles = data['data']['dataset']['datasetProfiles'][0]['fieldProfiles']
df = pd.DataFrame(field_profiles)

# Rename the columns
df_fin = df.rename(columns={'fieldPath': 'column_name', 'min': 'min_val', 'max': 'max_val'})[['column_name', 'min_val', 'max_val']]

df_fin

Now we can do the same for sample values.

In [ ]:
import pandas as pd

data = response.json()

# Extract the data
field_profiles = data['data']['dataset']['datasetProfiles'][0]['fieldProfiles']
df = pd.DataFrame(field_profiles)

# Rename the columns
df_fin = df.rename(columns={'fieldPath': 'column_name', 'sampleValues': 'sample_values'})[['column_name', 'sample_values']]

df_fin

In [ ]:
# Getting a specific value

print(df_fin.loc[40, 'sample_values'])


In [ ]:
# Getting the count of how many sample values there are in each list

df_fin['sample_size'] = df_fin['sample_values'].apply(lambda x: len(x) if x is not None else 0)


In [ ]:
df_fin